# IterativeEikonal
Approximate the viscosity solution of the  Eikonal equation 
$$\begin{cases} \Vert \nabla_G W(g) \Vert = 1, & g \in G \setminus S, \\
W(g) = 0, & g \in S \subset G. \end{cases}$$
This method is based on the paper ["A PDE Approach to Data-Driven Sub-Riemannian Geodesics in $SE(2)$" (2015) by E. J. Bekkers, R. Duits, A. Mashtakov, and G. R. Sanguinetti](https://doi.org/10.1137/15M1018460).

In [ ]:
import iterativeeikonal as eik
import taichi as ti
ti.init(arch=ti.gpu, debug=False)
import numpy as np
import scipy as sp
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib widget

## $\mathbb{R}^2$

### Flat space
In fact, we will solve the Eikonal equation on some subset $G \subset \mathbb{R}^2$, namely $G = [-1, 1] \times [-1, 1]$. The source set will consist of a single point: $S = \{(0, 0)\}$. Then we know that the viscosity solution is simply the Euclidean norm. 

There are numerous ways to numerically solve the Eikonal equation. We could use [Fast Marching](https://en.wikipedia.org/wiki/Fast_marching_method). This is an efficient method, but it requires a lot of work to make it compatible with non-Euclidean domains. 

In [ ]:
def fast_marching_R2(f):
    """
    Naive implementation of fast marching to solve Eikonal equation on R2 with 
    cost `f`."""
    N = f.shape[0]
    f = pad_array(f, 1.)
    dxy = 2. / N
    point_stages = 2 * np.ones((N, N), dtype=int)
    W = np.full(shape=(N, N), fill_value=100.)
    point_stages = pad_array(point_stages, 0)
    W = pad_array(W, 100.)
    i_0, j_0 = ((N + 1) // 2, (N + 1) // 2)
    point_stages[i_0, j_0] = 0
    W[i_0, j_0] = 0
    i_star, j_star = i_0, j_0
    while np.any(point_stages != 0):
        update_neighbours(i_star, j_star, point_stages, W, dxy, f)
        Trial = point_stages == 1
        index = np.argmin(np.where(Trial, W, np.inf))
        i_star, j_star = np.unravel_index(index, W.shape)
        point_stages[i_star, j_star] = 0
    return unpad_array(W)

def pad_array(array, fill_value):
    padded_shape = (array.shape[0] + 2, array.shape[1] + 2)
    padded_array = np.ones(padded_shape, dtype=array.dtype) * fill_value
    padded_array[1:-1, 1:-1] = array
    return padded_array

def unpad_array(padded_array):
    return padded_array[1:-1, 1:-1]

def update_neighbours(i_star, j_star, point_stages, W, dxy, f):
    neighbours = sees_point(i_star, j_star)
    for i, j in neighbours:
        if point_stages[i, j] != 0:
            point_stages[i, j] = 1
            update_W(i, j, W, dxy, f)

def update_W(i, j, W, dxy, f):
    Wx, Wy = gradient_W(i, j, W)
    if np.abs(Wx - Wy) >= dxy:
        W[i, j] = np.min((Wx + dxy / f[i, j], Wy + dxy / f[i, j]))
    else:
        W[i, j] = (Wx + Wy + np.sqrt((Wx + Wy) ** 2 - 2 * (Wx ** 2 + Wy ** 2 - (dxy / f[i, j]) ** 2))) / 2

def gradient_W(i, j, W):
    Wx = min(W[i + 1, j], W[i - 1, j])
    Wy = min(W[i, j + 1], W[i, j - 1])
    return Wx, Wy

def sees_point(i_star, j_star):
    return ((i_star + 1, j_star), (i_star - 1, j_star), (i_star, j_star + 1), (i_star, j_star - 1))

We can alternatively apply an iterative method, developed by [Bekkers et al.](https://doi.org/10.1137/15M1018460)

In [ ]:
def iterative_method_flat_R2(N, n):
    """
    Discretise [-1, 1] x [-1, 1] into `N` points in each direction, and apply 
    the iterative solution method `n` times.
    """
    dxy = 2. / (N + 1)
    eps = dxy / 4

    W = get_initial_W(N)
    boundarypoints, boundaryvalues = get_boundary_conditions(N)

    eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy = get_initial_derivatives(W)
    for _ in tqdm(range(n)):
        step_W(W, dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy, dxy, eps)
        eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    W_np = W.to_numpy()
    return eik.cleanarrays.unpad_array(W_np)

def get_initial_W(N, initial_condition=100.):
    W_unpadded = np.full(shape=(N, N), fill_value=initial_condition)
    W_np = eik.cleanarrays.pad_array(W_unpadded, pad_value=initial_condition, pad_shape=1)
    W = ti.field(dtype=ti.f32, shape=W_np.shape)
    W.from_numpy(W_np)
    return W

def get_boundary_conditions(N):
    i_0, j_0 = (N + 1) // 2, (N + 1) // 2
    boundarypoints_np = np.array([[i_0, j_0]], dtype=int)
    boundaryvalues_np = np.array([0.], dtype=float)
    boundarypoints = ti.Vector.field(n=2, dtype=ti.i32, shape=1)
    boundarypoints.from_numpy(boundarypoints_np)
    boundaryvalues = ti.field(shape=1, dtype=ti.f32)
    boundaryvalues.from_numpy(boundaryvalues_np)
    return boundarypoints, boundaryvalues

def get_initial_derivatives(W):
    dx_forward = ti.field(dtype=ti.f32, shape=W.shape)
    dx_backward = ti.field(dtype=ti.f32, shape=W.shape)
    dy_forward = ti.field(dtype=ti.f32, shape=W.shape)
    dy_backward = ti.field(dtype=ti.f32, shape=W.shape)
    abs_dx = ti.field(dtype=ti.f32, shape=W.shape)
    abs_dy = ti.field(dtype=ti.f32, shape=W.shape)
    return dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy

@ti.kernel
def step_W(
    W:ti.template(), 
    dx_forward: ti.template(), 
    dx_backward: ti.template(), 
    dy_forward: ti.template(), 
    dy_backward: ti.template(), 
    abs_dx: ti.template(), 
    abs_dy: ti.template(), 
    dxy: ti.f32, 
    eps: ti.f32
):
    eik.derivativesR2.abs_derivatives(W, dxy, dx_forward, dx_backward, dy_forward, dy_backward, abs_dx, abs_dy)
    for I in ti.grouped(W):
        W[I] += (1 - ti.math.sqrt(abs_dx[I] ** 2 + abs_dy[I] ** 2)) * eps

In [ ]:
N = 51
n = 250
xs, ys = np.meshgrid(np.linspace(-1, 1, N), np.linspace(-1, 1, N))
W_exact = np.sqrt(xs ** 2 + ys ** 2)
W_R2_flat = iterative_method_flat_R2(N, n)
f = np.ones((N, N), dtype=float)
W_fast_marching = fast_marching_R2(f)

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
contour = ax.contour(xs, ys, W_fast_marching, linestyles="dotted")
contour = ax.contour(xs, ys, W_R2_flat, linestyles="dashed")
ax.contour(xs, ys, W_exact)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_xlabel("x")
ax.set_ylabel("y")
fig.colorbar(contour, label="W(x, y)");

### Retinal image
Computing the cost function on flat $\mathbb{R}^2$ is not very impressive. We would like to be able to do it also when the space is not flat, for instance in the case of a retinal image. To track the vessels in a retinal image, it makes sense to assign a low cost to those areas of the image that are vessels, and a high cost to those that aren't. We must therefore first compute such a cost function.

To do this, we use Frangi vesselness filtering, developed by Frangi et al. in ["Multiscale vessel enhancement filtering"](https://doi.org/10.1007/BFb0056195) (1998).

#### Frangi vesselness filtering
Let's first load in the image.

In [ ]:
ds = 8
retinal_image = Image.open("E46_OD_best.tif")
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_unnormalised = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array = eik.cleanarrays.image_rescale(retinal_array_unnormalised)
eik.cleanarrays.view_image_array(retinal_array);

We will already define the start and the end of our geodesic (`source_point` and `target_point`, respectively).

In [ ]:
source_point = (246, 302) # "y", "x" so row, column.
target_point = (211, 118)
# target_point = (194, 250)
i_min, i_max = 0, retinal_array.shape[0] - 1
j_min, j_max = 0, retinal_array.shape[0] - 1
xs, ys = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1))

We can now perform the Frangi filtering. The values for the scales, as well as the Frangi filter parameters $\alpha$, $\gamma$, and $\varepsilon$ were taken from the Mathematica notebook "Code A - Vesselness in SE(2).nb".

In [ ]:
scales = (np.array((2, 3, 4, 5), dtype=float)) / 4
vesselness_unmasked = eik.multiscale_frangi_filter_R2(-retinal_array, scales, α=0.2, γ=3/4, ε=0.2)
mask = (retinal_array > 0) # Remove boundary
vesselness_unnormalised = vesselness_unmasked * sp.ndimage.binary_erosion(mask, iterations=int(np.ceil(scales.max() * 2)))
print(f"Before rescaling, vesselness is in [{vesselness_unnormalised.min()}, {vesselness_unnormalised.max()}].")
vesselness = eik.cleanarrays.image_rescale(vesselness_unnormalised)

In [ ]:
image_vesselness = eik.cleanarrays.convert_array_to_image(vesselness)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.imshow(image_vesselness, cmap="gray", origin="upper")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

With these parameters, the vesselness score produced by the Frangi filter looks similar to the one in the Mathematica notebook "code A - Vesselness in SE(2).nb". However, the normalisation is not the same.

#### Cost function
Given the vesselness function, we compute the cost function as
$$\mathrm{cost}_{\lambda, p}(x, y) = \frac{1}{1 + \lambda \cdot \vert \mathrm{vesselness} \vert^p}$$
We choose $p = 2$, like in the Mathematica notebook, but $\lambda = 100$ instead of $\lambda = 1000$. This produces a cost function that looks more similar (probably due to the difference in normalisation).

In [ ]:
cost = eik.cost_function(vesselness, 100, 2)

In [ ]:
image_cost = eik.cleanarrays.convert_array_to_image(cost)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.imshow(image_cost, cmap="gray", origin="upper")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

#### Distance map
We are now able to compute the distance map, which is the viscosity solution of the Eikonal equation
$$\begin{cases} \Vert \nabla_{\mathrm{cost}} W(x, y) \Vert = 1, & (x, y) \in \mathbb{R}^2 \setminus \{(x_0, y_0)\}, \\
W(x, y) = 0, & (x, y) = (x_0, y_0), \end{cases}$$
where $\nabla_{\mathrm{cost}}$ is a datadriven derivative:
$$\Vert \nabla_{\mathrm{cost}} W(x, y) \Vert = \sqrt{\mathrm{cost}_{\lambda, p}^{-2}(x, y) (\vert \partial_x W(x, y) \vert^2 + \vert \partial_y W(x, y) \vert^2)}.$$
The quality of the approximation spreads out from the source point. Hence, if we are interested only in the distance map to points that are not that far away (in terms of the true distance map), then we do not have to perform as many iterations. __We terminate the iteration when the value of the approximate distance map changes less than a certain tolerance ($10^{-5}$) after a step. NOT ANYMORE__

In [ ]:
W_R2_retinal, grad_W_R2 = eik.eikonal_solver_R2(cost, source_point)
# W_fast_marching = fast_marching_R2(cost) # Too slow...

If all is well, then the distance map should be $0$ at the source point, and vary nicely around the source_point.

In [ ]:
W_R2_retinal[source_point[0]-3:source_point[0]+2, source_point[1]-3:source_point[1]+2]

We can now visualise the distance map with a contour plot.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax.imshow(cost, cmap="gray", origin="upper")
max_distance = np.round(W_R2_retinal[target_point] * 2.5)
contour = ax.contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend()
fig.colorbar(contour, label="W[i, j]");

We have also computed the gradient field $\nabla_{\mathrm{cost}} W$, which is shown below.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax.imshow(vesselness, cmap="gray", origin="upper")
ax.streamplot(xs[100:-100, 100:-100], ys[100:-100, 100:-100], -grad_W_R2[100:-100, 100:-100, 1], -grad_W_R2[100:-100, 100:-100, 0], color="red")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

### Geodesic tracking
Now that we have the distance map (from a certain source point), we are able to compute the geodesics between the source point and any other point $(x^*, y^*)$ using backtracking:
$$\begin{dcases} \dot{\gamma}(t) = -\left(\frac{\partial_x W(\gamma(t))}{\mathrm{cost}_{\lambda, p}(\gamma(t))}, \frac{\partial_y W(\gamma(t))}{\mathrm{cost}_{\lambda, p}(\gamma(t))}\right), & t > 0, \\
\gamma(0) = (x^*, y^*). & \end{dcases}$$
We can numerically solve this using a Forward Euler discretisation:
$$\begin{dcases} \gamma_{n + 1} = \gamma_n - \Delta t \left(\frac{\partial_x W(\gamma_n)}{\mathrm{cost}_{\lambda, p}(\gamma_n)}, \frac{\partial_y W(\gamma_n)}{\mathrm{cost}_{\lambda, p}(\gamma_n)}\right), & n > 0, \\
\gamma_0 = (x^*, y^*). & \end{dcases}$$

In [ ]:
np.sqrt(np.sum(grad_W_R2[200, 300, :]**2))

In [ ]:
W_R2_retinal[target_point]

In [ ]:
γ_ci = eik.geodesic_back_tracking_R2(grad_W_R2, source_point, target_point, dt=1., n_max=1000)
γ = eik.convert_continuous_indices_to_real_space_R2(γ_ci, xs, ys)

We can see from the visualisation below that the geodesic follows the vessel rather well. However, this vessel was quite easy to follow: there were no crossings where the backtracking in $\mathbb{R}^2$ could take shortcuts.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax.imshow(vesselness, cmap="gray")
# contour = ax.contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax.plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(100, 320)
ax.set_ylim(340, 120)
ax.legend();
# fig.colorbar(contour, label="W[i, j]");

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax[0].imshow(vesselness, cmap="gray")
contour = ax[0].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax[0].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[0].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[0].set_xlabel("j")
ax[0].set_ylabel("i")
ax[0].set_xlim(j_min, j_max)
ax[0].set_ylim(i_max, i_min)
ax[0].legend()
fig.colorbar(contour, label="W[i, j]")

ax[1].imshow(vesselness, cmap="gray")
contour = ax[1].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax[1].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[1].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[1].set_xlabel("j")
ax[1].set_ylabel("i")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(340, 120)
ax[1].legend()
fig.colorbar(contour, label="W[i, j]");

In [ ]:
target_point_bad = (199, 179)
γ_ci_bad = eik.geodesic_back_tracking_R2(grad_W_R2, source_point, target_point_bad, dt=1., n_max=1000)
γ_bad = eik.convert_continuous_indices_to_real_space_R2(γ_ci_bad, xs, ys)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax[0].imshow(vesselness, cmap="gray")
contour = ax[0].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ_bad[:, 0], γ_bad[:, 1], label="geodesic", color="red")
ax[0].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[0].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[0].set_xlabel("j")
ax[0].set_ylabel("i")
ax[0].set_xlim(j_min, j_max)
ax[0].set_ylim(i_max, i_min)
ax[0].legend()
fig.colorbar(contour, label="W[i, j]")

ax[1].imshow(vesselness, cmap="gray")
contour = ax[1].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ_bad[:, 0], γ_bad[:, 1], label="geodesic", color="red")
ax[1].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[1].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[1].set_xlabel("j")
ax[1].set_ylabel("i")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(340, 120)
ax[1].legend()
fig.colorbar(contour, label="W[i, j]");

## $SE(2)$
We have now done vessel tracking in flat Euclidean space. Since $\mathbb{R}^2$ is isotropic, moving in any direction (before introducing a datadriven cost term) is equally expensive. Vessels, however, have a clear orientation - namely in the direction of the vessel - which typically does not drastically change as we move along the vessel. It therefore would make sense to penalise motions that are not in line with the vessel orientation. We can do this by modelling the retina with the special Euclidean group $SE(2) = \mathbb{R}^2 \rtimes S^1$, which can be interpreted as the space of positions and orientations on $\mathbb{R}^2$. 

### $SE(2)$ background
The space of positions and orientations is a Lie group, which means it is a smooth manifold with a group product. The group product on $SE(2)$ is given by
$$(\vec{x}_1, \theta_1) \cdot (\vec{x}_2, \theta_2) = (\vec{x}_1 + R_{\theta_1} \vec{x}_2, \theta_1 + \theta_2),$$
where 
$$R_{\theta} = \begin{pmatrix} \cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \end{pmatrix}$$
is the matrix corresponding to a rotation with angle $\theta$.

In fact, $\mathbb{R}^2$ is also a Lie group, with $+$ as group operation. We can identify derivatives at a point in $\mathbb{R}^2$ with vectors in the tangent space at that point. Of particular interest are left invariant derivatives, which are exactly those that commute with left actions. The left action operator $\mathcal{L}: G \times L^2(G) \to L^2(G)$ on a Lie group $G$ is defined by
$$\mathcal{L}_g U(h) = U(L_g^{-1} h) = U(g^{-1} h)$$
for all $g, h \in G$. On $\mathbb{R}^2$, this would look like
$$\mathcal{L}_{\vec{y}} U(\vec{x}) = U(\vec{x} - \vec{y}).$$
Then a differential operator $\mathcal{A}$ commutes with left actions if 
$$\mathcal{L}_g \circ \mathcal{A} = \mathcal{A} \circ \mathcal{L}_g$$
for all $g \in G$, or, more concretely, if
$$(\mathcal{L}_g \circ \mathcal{A})(U)(h) = (\mathcal{A} \circ \mathcal{L}_g)(U)(h)$$
for all $g \in G$ and all sufficiently smooth $U \in L^2(G)$. It can be shown that on $\mathbb{R}^2$, the left invariant derivatives are (constant) linear combinations of $\partial_x$ and $\partial_y$. On $SE(2)$, however, things are a bit different: there the left invariant derivatives are linear combinations of the following vector fields:
$$\begin{align*} \mathcal{A}_1|_{(\vec{x}, \theta)} & = \cos(\theta) \partial_x|_{(\vec{x}, \theta)} + \sin(\theta) \partial_y|_{(\vec{x}, \theta)}, \\
\mathcal{A}_2|_{(\vec{x}, \theta)} & = -\sin(\theta) \partial_x|_{(\vec{x}, \theta)} + \cos(\theta) \partial_y|_{(\vec{x}, \theta)}, \textrm{ and} \\
\mathcal{A}_3|_{(\vec{x}, \theta)} & = \partial_{\theta}|_{(\vec{x}, \theta)}. \end{align*}$$
$\mathcal{A}_1$ and $\mathcal{A}_2$, which describe derivatives in the spatial directions, now depend on the orientation $\theta$.

To finally define the gradient on $SE(2)$, we need one more component: the metric tensor (field) $\mathcal{G}$. At each point, the metric tensor defines an inner product on the tangent space. We are again interested in the left invariant versions. It then turns out that then the metric tensor may be written as
$$\mathcal{G}_{(\vec{x}, \theta)} = \sum_{i = 1}^3 \sum_{j = 1}^3 g_{ij} \omega^i|_{(\vec{x}, \theta)} \otimes \omega^j|_{(\vec{x}, \theta)},$$
where $\omega^i$ is the dual of $\mathcal{A}_i$ (so $\omega^i(\mathcal{A}_j) = \delta_j^i$), and $g_{ij}$ is constant.

### Flat space
To get a feel for it, we will again start by computing the distance map on flat space, when the cost function is identically equal to $1$. (Now with plus controller)

In [ ]:
def iterative_method_flat_SE2(N, N_ors, n, plus_controller=False):
    """
    Discretise [-1, 1] x [-1, 1] x [-π, π) into `N` points in each spatial 
    direction and `N_ors` in the orientational direction, and apply the 
    iterative solution method `n` times.
    """
    dxy = 2. / (N + 1)
    ε = dxy / 4
    W = get_initial_W_SE2(N, N_ors)

    # Create empty Taichi objects
    A1_forward = ti.field(dtype=ti.f32, shape=W.shape)
    A1_backward = ti.field(dtype=ti.f32, shape=W.shape)
    A2_forward = ti.field(dtype=ti.f32, shape=W.shape)
    A2_backward = ti.field(dtype=ti.f32, shape=W.shape)
    A3_forward = ti.field(dtype=ti.f32, shape=W.shape)
    A3_backward = ti.field(dtype=ti.f32, shape=W.shape)
    A1_W = ti.field(dtype=ti.f32, shape=W.shape)
    A2_W = ti.field(dtype=ti.f32, shape=W.shape)
    A3_W = ti.field(dtype=ti.f32, shape=W.shape)

    boundarypoints, boundaryvalues = get_boundary_conditions(N, N_ors)
    eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    # Compute approximate distance map
    if plus_controller:
        for _ in tqdm(range(n)):
            step_W_SE2_plus(W, A1_forward, A1_backward, A2_forward, A2_backward, A3_forward, A3_backward, A1_W, A2_W, 
                            A3_W, dxy, ε)
            eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)
    else:
        for _ in tqdm(range(n)):
            step_W_SE2(W, A1_forward, A1_backward, A2_forward, A2_backward, A3_forward, A3_backward, A1_W, A2_W, A3_W, 
                       dxy, ε)
            eik.cleanarrays.apply_boundary_conditions(W, boundarypoints, boundaryvalues)

    W_np = W.to_numpy()
    return eik.cleanarrays.unpad_array(W_np)

def get_initial_W_SE2(N, N_ors, initial_condition=100.):
    W_unpadded = np.full(shape=(N, N, N_ors), fill_value=initial_condition)
    W_np = eik.cleanarrays.pad_array(W_unpadded, pad_value=initial_condition)
    W = ti.field(dtype=ti.f32, shape=W_np.shape)
    W.from_numpy(W_np)
    return W

def get_boundary_conditions(N, N_ors):
    i_0, j_0, θ_0 = (N - 1) // 2, (N - 1) // 2, N_ors // 2
    boundarypoints_np = np.array([[i_0 + 1, j_0 + 1, θ_0 + 1]], dtype=int) # Account for padding.
    boundaryvalues_np = np.array([0.], dtype=float)
    boundarypoints = ti.Vector.field(n=3, dtype=ti.i32, shape=1)
    boundarypoints.from_numpy(boundarypoints_np)
    boundaryvalues = ti.field(shape=1, dtype=ti.f32)
    boundaryvalues.from_numpy(boundaryvalues_np)
    return boundarypoints, boundaryvalues

def get_boundary_conditions_PT(N, N_ors):
    i_0, j_0, θ_0 = (N - 1) // 2, (N - 1) // 2, N_ors // 2
    i_1, j_1, θ_1 = (N - 1) // 2, (N - 1) // 2, 3 * N_ors // 4
    i_2, j_2, θ_2 = (N - 1) // 2, (N - 1) // 2, 1 * N_ors // 4
    boundarypoints_np = np.array([[i_0, j_0, θ_0], [i_1, j_1, θ_1], [i_2, j_2, θ_2]], dtype=int)
    boundaryvalues_np = np.array([0., 0., 0.], dtype=float)
    boundarypoints = ti.Vector.field(n=3, dtype=ti.i32, shape=3)
    boundarypoints.from_numpy(boundarypoints_np)
    boundaryvalues = ti.field(shape=3, dtype=ti.f32)
    boundaryvalues.from_numpy(boundaryvalues_np)
    return boundarypoints, boundaryvalues

@ti.kernel
def step_W_SE2(
    W:ti.template(),
    A1_forward: ti.template(),
    A1_backward: ti.template(),
    A2_forward: ti.template(),
    A2_backward: ti.template(),
    A3_forward: ti.template(),
    A3_backward: ti.template(), 
    A1_W: ti.template(),
    A2_W: ti.template(),
    A3_W: ti.template(),
    dxy: ti.f32,
    ε: ti.f32
):
    eik.derivativesSE2.derivatives_LI(W, dxy, A1_forward, A1_backward, A2_forward, A2_backward, A3_forward, A3_backward)
    eik.derivativesSE2.upwind_derivatives_LI(W, dxy, A1_forward, A1_backward, A2_forward, A2_backward, A3_forward,
                                             A3_backward, A1_W, A2_W, A3_W)
    for I in ti.grouped(W):
        W[I] += (1 - ti.math.sqrt(0.05 * A1_W[I]**2 + 0 * A2_W[I]**2 + A3_W[I]**2)) * ε

@ti.kernel
def step_W_SE2_plus(
    W:ti.template(),
    A1_forward: ti.template(),
    A1_backward: ti.template(),
    A2_forward: ti.template(),
    A2_backward: ti.template(),
    A3_forward: ti.template(),
    A3_backward: ti.template(), 
    A1_W: ti.template(),
    A2_W: ti.template(),
    A3_W: ti.template(),
    dxy: ti.f32,
    ε: ti.f32
):
    eik.derivativesSE2.derivatives_LI(W, dxy, A1_forward, A1_backward, A2_forward, A2_backward, A3_forward, A3_backward)
    eik.derivativesSE2.upwind_derivatives_LI(W, dxy, A1_forward, A1_backward, A2_forward, A2_backward, A3_forward,
                                             A3_backward, A1_W, A2_W, A3_W)
    for I in ti.grouped(W):
        W[I] += (1 - ti.math.sqrt(0.05 * ti.math.max(A1_W[I], 0)**2 + A3_W[I]**2)) * ε

In [ ]:
N = 201
N_ors = 32
n = 5000
xs, ys, θs = np.meshgrid(np.linspace(-1, 1, N), np.linspace(-1, 1, N), np.linspace(-1 * np.pi, 1 * np.pi, N_ors))
W_SE2_flat = iterative_method_flat_SE2(N, N_ors, n)
print(W_SE2_flat.min(), W_SE2_flat.mean(), W_SE2_flat.max())
print(W_SE2_flat[100:103, 100:103, 15:18])
W_SE2_flat_plus = iterative_method_flat_SE2(N, N_ors, 4 * n, plus_controller=True)
print(W_SE2_flat_plus.min(), W_SE2_flat_plus.mean(), W_SE2_flat_plus.max())
print(W_SE2_flat_plus[100:103, 100:103, 15:18])

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.linspace(-1, 1, N), W_SE2_flat[:, (N - 1) // 2, N_ors // 2], label="Sub Riemannian")
ax.plot(np.linspace(-1, 1, N), W_SE2_flat_plus[:, (N - 1) // 2, N_ors // 2], label="Plus Controller")
ax.set_xlim(-1, 1)
ax.set_ylim(0, 5)
ax.set_title("Comparison of Distance Maps Along $x$-Axis")
ax.legend();

To visualise the distance map, we will look at isosurfaces, i.e., surfaces in $SE(2)$ such that the distance is constant over the surface. Mesh approximations of these can be computed using the marching cubes algorithm, implemented e.g. in [scikit-image](https://github.com/scikit-image/scikit-image).

In [ ]:
from skimage.measure import marching_cubes, block_reduce
# Maybe use Plotly or Mayavi instead of matplotlib

In [ ]:
verts_flat, faces_flat, normals_flat, values_flat = marching_cubes(W_SE2_flat, level=4, step_size=4)
print(values_flat.min(), values_flat.mean(), values_flat.max(), values_flat.shape)
verts_flat_plus, faces_flat_plus, normals_flat_plus, values_flat_plus = marching_cubes(W_SE2_flat_plus, level=4, step_size=4)
print(values_flat_plus.min(), values_flat_plus.mean(), values_flat_plus.max(), values_flat_plus.shape)

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf((2 * verts_flat[:, 0] / N) - 1, (2 * verts_flat[:, 1] / N) - 1, faces_flat, (2 * np.pi * verts_flat[:, 2] / N_ors) - np.pi, label="Sub Riemannian", alpha=0.5)
ax.plot_trisurf((2 * verts_flat_plus[:, 0] / N) - 1, (2 * verts_flat_plus[:, 1] / N) - 1, faces_flat_plus, (2 * np.pi * verts_flat_plus[:, 2] / N_ors) - np.pi, label="Plus Controller", alpha=0.5)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-np.pi, np.pi)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("θ")
ax.legend();

__GPU MEMORY STUFF__
GPU has 4 GB of memory (20 GB shared). I think TaiChi crashes if we want to use more. Each number is 4 bytes, imported cost function is 512 x 512 x 32 numbers so 0.5 GB. At any one time we have circa 15 objects of that size to store intermediate results, so that is about 7.5 GB.

If we downsample by a factor 2 in each direction, we only use about 1 GB.

In [ ]:
cost_big = np.load("costOR.npy") # + 0.01
cost = block_reduce(cost_big, block_size=(2, 2, 2), func=np.min)
source_point = np.unravel_index(cost.argmin(axis=None), cost.shape)

_, _, ax = eik.cleanarrays.view_image_array(cost.min(axis=-1))
ax.scatter(source_point[1], source_point[0], marker=".");

In [ ]:
G_inv = np.array(((0.05, 0., 0.), (0., 0., 0.), (0., 0., 1.)))
W_SE2_py, grad_W_SE2_py = eik.solvers.eikonal_solver_SE2_LI(G_inv, cost, source_point, 1., n_max=50000)
W_SE2_py.min(), W_SE2_py.mean(), W_SE2_py.max()

In [ ]:
print(W_SE2_py[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2, source_point[2]-1:source_point[2]+2])
W_SE2_min = W_SE2_py.min(axis=-1)
print(W_SE2_min[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2])

In [ ]:
_, _, ax = eik.cleanarrays.view_image_array(W_SE2_min / W_SE2_min.max());
ax.scatter(source_point[1], source_point[0], marker=".");

In [ ]:
i_min, i_max = 0, W_SE2_min.shape[0] - 1
j_min, j_max = 0, W_SE2_min.shape[1] - 1
xs, ys = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax.imshow(cost.min(axis=-1), cmap="gray", origin="upper")
# max_distance = np.round(W_R2_retinal[target_point] * 2.5)
contour = ax.contour(xs, ys, W_SE2_min) # , levels=np.linspace(0., max_distance, 5))
ax.scatter(xs[source_point[:2]], ys[source_point[:2]], label="Source", marker=".")
# ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend()
fig.colorbar(contour, label="W[i, j]");

In [ ]:
# grad_W_SE2_py.min(axis=-2).shape

In [ ]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
# fig.canvas.toolbar_visible = False
# fig.canvas.header_visible = False
# fig.canvas.footer_visible = False
# fig.canvas.resizable = False
# ax.imshow(vesselness, cmap="gray", origin="upper")
# ax.streamplot(xs[100:-100, 100:-100], ys[100:-100, 100:-100], -grad_W_R2[100:-100, 100:-100, 1], -grad_W_R2[100:-100, 100:-100, 0], color="red")
# ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
# ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
# ax.set_xlabel("j")
# ax.set_ylabel("i")
# ax.set_xlim(j_min, j_max)
# ax.set_ylim(i_max, i_min)
# ax.legend();

In [ ]:
verts_py, faces_py, normals_py, values_py = marching_cubes(W_SE2_py, level=50, step_size=7)
values_py.min(), values_py.mean(), values_py.max(), values_py.shape

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf((2 * verts_py[:, 0] / N) - 1, (2 * verts_py[:, 1] / N) - 1, faces_flat, (2 * np.pi * verts_py[:, 2] / N_ors) - np.pi, label="Sub Riemannian", alpha=0.5)
ax.set_zlim(-np.pi, np.pi)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("θ")
ax.legend();

### Retinal image
We would now like to process the image of the retina in $SE(2)$, since thereon crossing vessels will become untangled.

#### Vesselness
Let's first load in the image, and apply a high pass filter

In [ ]:
ds = 8
retinal_image = Image.open("E46_OD_best.tif")
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_gray_ds = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array_unnormalised = eik.cleanarrays.high_pass_filter(retinal_array_gray_ds, 2 * 32 / ds)
retinal_array = eik.cleanarrays.image_rescale(retinal_array_unnormalised)
eik.cleanarrays.view_image_array(retinal_array);

We will already define the start and the end of our geodesic (`source_point` and `target_point`, respectively).

In [ ]:
def convert_angle_to_index(θ, N_ors):
    return int(N_ors * (θ + np.pi) / (2 * np.pi)) // N_ors

In [ ]:
N_ors = 65
θ_source = -5 * np.pi / 8
θ_target = np.pi
source_point = (246, 302, convert_angle_to_index(θ_source, N_ors)) # "y", "x" so row, column.
target_point = (211, 118, convert_angle_to_index(θ_target, N_ors))
# target_point = (194, 250)
i_min, i_max = 0, retinal_array.shape[0] - 1
j_min, j_max = 0, retinal_array.shape[0] - 1
θ_min, θ_max = -np.pi, np.pi
xs, ys, θs = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1), np.linspace(-np.pi, np.pi, N_ors))

We can now perform the Frangi filtering. The values for the scales, as well as the Frangi filter parameters $\alpha$, $\gamma$, and $\varepsilon$ were taken from the Mathematica notebook "Code A - Vesselness in SE(2).nb".

In [ ]:
scales = (np.array((2, 3, 4, 5), dtype=float)) / 4
vesselness_unmasked = eik.multiscale_frangi_filter_R2(-retinal_array, scales, α=0.2, γ=3/4, ε=0.2)
mask = (retinal_array_gray_ds > 0) # Remove boundary
vesselness_unnormalised = vesselness_unmasked * sp.ndimage.binary_erosion(mask, iterations=int(np.ceil(scales.max() * 2)))
print(f"Before rescaling, vesselness is in [{vesselness_unnormalised.min()}, {vesselness_unnormalised.max()}].")
vesselness = eik.cleanarrays.image_rescale(vesselness_unnormalised)

In [ ]:
image_vesselness = eik.cleanarrays.convert_array_to_image(vesselness)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.imshow(image_vesselness, cmap="gray", origin="upper")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".", color="blue")
ax.arrow(xs[source_point], ys[source_point], 20 * np.cos(θ_source), 20 * np.sin(θ_source), color="blue", head_width=5)
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".", color="red")
ax.arrow(xs[target_point], ys[target_point], 20 * np.cos(θ_target), 20 * np.sin(θ_target), color="red", head_width=5)
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

With these parameters, the vesselness score produced by the Frangi filter looks similar to the one in the Mathematica notebook "code A - Vesselness in SE(2).nb". However, the normalisation is not the same.